In [1]:
import numpy as np

In [2]:
# input parameters
#--sham <in.sham> --out <out.tsv> \
#--p01 <0 to 1 error rate> --p10 <1 to 0 error rate> \
#--prior1 <prior probability that each base is a 1>
sham = 'tiny_example.sham'
out = 'variant_calling.tsv'
p01 = 0.1
p10 = 0.1
prior1 = 0.5

In [3]:
# Function for reading sham file
def read_sham_file(sham):
    with open (sham, 'r') as shamfile:
        data = shamfile.readlines()
    locations = []
    reads = []
    for d in data:
        rd = d.split('\t')
        locations.append(int(rd[0]))
        reads.append(rd[1].rstrip())
    return locations, reads

In [4]:
# Function for writing the result
def write_result(out, posterior_prob, max_len):
    with open (out, 'w') as out_file:
        for i in range(max_len):
            out_file.write("{}\t{}\n".format(i, format(posterior_prob[i], '.3f')))	

In [5]:
# Reading the input file
locations, reads = read_sham_file(sham)

In [6]:
# Calculating the max size of the predicted string to pre-alocate the arrays
max_len = max([locations[i]+len(reads[i]) for i in range(len(reads))])

In [7]:
# Number of zeros at every location
zeros = np.zeros(max_len, int)
# Number of ones at every location
ones = np.zeros(max_len, int)

In [8]:
# Loop over reads to calculate the number of ones and zeros at every location
for i in range(len(reads)):
    rd_ones = np.array([int(n) for n in reads[i]])
    lc = locations[i] 
    ones[lc:lc+rd_ones.shape[0]] += rd_ones 
    zeros[lc:lc+rd_ones.shape[0]] += (1-rd_ones)

In [9]:
# Pre-alocate the posterior probabilities
posterior_prob = np.zeros(max_len, float)

In [10]:
# Calculate the posterior probabilities at every location
for i in range(max_len):
    numerator = (p10**zeros[i]) * ((1-p10)**ones[i]) * prior1
    denominator = numerator + (p01**ones[i]) * ((1-p01)**zeros[i]) * (1-prior1)
    posterior_prob[i] = numerator / denominator

In [11]:
# Writing the result to the output file
write_result(out, posterior_prob, max_len)